# Temperature Vs. Probability of Failure

In [ ]:
%matplotlib inline
%matplotlib notebook

import numpy as np, math
import matplotlib.pyplot as plt

# Create sample dataset
xvalues = np.linspace(20,100, 100)
yvalues = np.array([((np.cos((x-50)/100)*100 + np.sin(x/100)*100 + np.random.normal(0, 13, 1)[0])/150.0) for x in xvalues])
ymin = np.amin(yvalues)
yvalues = [y - ymin for y in yvalues]

# Plot dataset
fig0, ax0 = plt.subplots()
plt.title("Engine Temp Vs. Probability of Failure")
plt.ylabel("Probability of Failure")
plt.xlabel("Engine Temp in Degrees Celcius")
ax0.plot(xvalues,
        yvalues, "ro", linewidth = 2, color = 'red', label = 'target')
ax0.set_xlim(20, 100)
ax0.set_ylim(0, 1)


# Build a Model to Fit The Data

In [ ]:
%matplotlib inline
%matplotlib notebook

#----------
# build the dataset
#----------
from pybrain.datasets import SupervisedDataSet
import numpy as np, math
from nn_utils import train_nn

ds = SupervisedDataSet(1, 1)
for x, y in zip(xvalues, yvalues):
    ds.addSample((x,), (y,))

#----------
# build the network
#----------
net = train_nn(xvalues, yvalues, rate=0.0001, layers = [100,100,100], batch=False, magnification=0.0, iterations = 50)
    

# Use Model In Realtime Streaming Application

In [ ]:
from streamsx.topology.topology import Topology
from my_module import array_yielder, neural_net_model
from streamsx.topology import context
from jupyter_utilities import get_next_line, graph_from_outstream
import time, sys
import matplotlib.pyplot as plt
import random

# Define test data
xdata_raw = [random.uniform(0,100) for x in range(100)]

# Define operators
ay = array_yielder(xdata_raw)
nnm = neural_net_model(net)

# Build Graph
top = Topology("myTop")
stream_ = top.source(ay)
stream_.transform(nnm).print()
# Submit
out= context.submit("JUPYTER", top.graph)

graph_from_outstream(out, xdata_raw, 20, 100, 0, 1)